## 16-04. RAG + Image Generator Agent(보고서 작성)
- 웹 검색(Web Search), PDF 문서 기반 검색(RAG), 이미지 생성(Image Generation) 등을 통해 보고서를 작성하는 에이전트

<br>



In [1]:
from dotenv import load_dotenv

load_dotenv()

True

<br>

#### 웹 검색도구: Tavily Search



In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
search = TavilySearchResults(k=6)

C:\Users\user\AppData\Local\Temp\ipykernel_20340\3100324165.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(k=6)


<br>

#### 문서 기반 문서 검색 도구: Retriever


In [4]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_classic.document_loaders import PyMuPDFLoader

In [5]:
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = loader.load_and_split(text_splitter)
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [6]:
from langchain_classic.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

- `document_prompt` 는 문서의 내용을 표시하는 템플릿을 정의

In [7]:
document_prompt = PromptTemplate.from_template(
    "<document><content>{page_content}</content><page>{page}</page><filename>{source}</filename></document>"
)

In [8]:
retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search for information in the PDF file",
    document_prompt=document_prompt,
)

<br>

#### DallE 이미지 생성 도구


In [9]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.tools import tool

In [10]:
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

In [11]:
@tool
def dalle_tool(query):
    """use this tool to generate image from text"""
    return dalle.run(query)

<br>

#### 파일 관리 도구


In [12]:
from langchain_community.agent_toolkits import FileManagementToolkit

In [13]:
working_directory = "tmp"

# 파일 관리 도구 생성(파일 쓰기, 읽기, 디렉토리 목록 조회)
file_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["write_file", "read_file", "list_directory"],
).get_tools()

file_tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp')]

<br>

#### 모든 도구를 종합

In [14]:
tools = file_tools + [
    retriever_tool,
    search,
    dalle_tool,
]

tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp'),
 StructuredTool(name='pdf_search', description='use this tool to search for information in the PDF file', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x00000228E2003740>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x00000228E349E160>),
 TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 StructuredTool(name='dalle_tool', description='use this tool to generate image from text', args_schema=<class 'langchain_core.utils.pydantic.dalle_tool'>, func=<function dalle_tool at 0x00000228E35242C0>)]

<br>

### Agent 생성

<br>

#### 1. 문서기반 검색 + 파일 생성


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI

In [16]:
store = {}

In [17]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "You are a professional researcher. "
            "You can use the pdf_search tool to search for information in the PDF file. "
            "You can find further information by using search tool. "
            "You can use image generation tool to generate image from text. "
            "Finally, you can use file management tool to save your research result into files.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [18]:
llm = ChatOpenAI(model="gpt-4o-mini")
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

In [19]:
def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]

In [20]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

<br>

- **에이전트 실행**

In [ ]:
result = agent_with_chat_history.stream(
    {
        "input": "삼성전자가 개발한 `생성형 AI` 와 관련된 유용한 정보들을 PDF 문서에서 찾아서 bullet point로 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 새롭게 생성하여 정리한 내용을 저장해주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 발췌한 PDF 문서의 페이지 번호, 파일명을 기입하세요(예시: page 10, filename.pdf). \n"
        "3. 정리된 bullet point를 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

<br>

#### 2. 웹 검색 + 파일 수정
- 생성된 보고서 파일 (`report.md`)의 내용을 업데이트


In [ ]:
result = agent_with_chat_history.stream(
    {
        "input": "이번에는 삼성전자가 개발한 `생성형 AI` 와 관련된 정보들을 웹 검색하고, 검색한 결과를 정리해 주세요. "
        "한글로 작성해주세요."
        "다음으로는 `report.md` 파일을 열어서 기존의 내용을 읽고, 웹 검색하여 찾은 정보를 이전에 작성한 형식에 맞춰 뒷 부분에 추가해 주세요. \n\n"
        "#작성방법: \n"
        "1. markdown header 2 크기로 적절한 제목을 작성하세요. \n"
        "2. 정보의 출처(url)를 기입하세요(예시: 출처: 네이버 지식백과). \n"
        "3. 정리된 웹검색 내용을 작성하세요. \n"
        "4. 작성이 완료되면 파일을 `report.md` 에 저장하세요. \n"
        "5. 마지막으로 저장한 `report.md` 파일을 읽어서 출력해 주세요. \n"
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

<br>

#### 3. 보고서 작성

In [ ]:
result = agent_with_chat_history.stream(
    {
        "input": "`report.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용을 바탕으로, 전문적인 수준의 보고서를 작성하세요. "
        "보고서는 총 3개의 섹션으로 구성되어야 합니다:\n"
        "1. 개요: 보고서 abstract 를 300자 내외로 작성하세요.\n"
        "2. 핵심내용: 보고서의 핵심 내용을 작성하세요. 정리된 표를 markdown 형식으로 작성하여 추가하세요. "
        "3. 최종결론: 보고서의 최종 결론을 작성하세요. 출처(파일명, url 등)을 표시하세요."
        "마지막으로 작성된 결과물을 `report-2.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

<br>

#### 보고서 내용을 기반으로 이미지 생성을 요청

In [ ]:
result = agent_with_chat_history.stream(
    {
        "input": "`report-2.md` 파일을 열어서 안의 내용을 출력하세요. "
        "출력된 내용에 어울리는 이미지를 생성하세요. "
        "생성한 이미지의 url 을 markdown 형식으로 보고서의 가장 상단에 추가하세요. "
        "마지막으로 작성된 결과물을 `report-3.md` 파일에 저장하세요."
    },
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")
for step in result:
    print(step)

<br>

<hr>

<br>

## 16-10. 도구를 활용한 토론 에이전트 (Two Agents Debates with Tools)

<br>

### `DialogueAgent` 및 `DialogueSimulator`

<br>

#### `DialogueAgent`
- `send` 메서드는 현재까지의 대화 기록과 에이전트의 접두사를 사용하여 채팅 모델에 메시지를 전달하고 응답을 반환
- `receive` 메서드는 다른 에이전트가 보낸 메시지를 대화 기록에 추가

In [22]:
from typing import Callable, List

from langchain_classic.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_openai import ChatOpenAI

In [ ]:
class DialogueAgent:
    def __init__(self, name: str, system_message: SystemMessage, model: ChatOpenAI) -> None:
        # 에이전트의 이름을 설정
        self.name = name
        # 시스템 메시지를 설정
        self.system_message = system_message
        # LLM 모델을 설정
        self.model = model
        # 에이전트 이름을 지정
        self.prefix = f"{self.name}: "
        # 에이전트를 초기화
        self.reset()

    def reset(self):
        """
        대화 내역을 초기화
        """
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        메시지에 시스템 메시지 + 대화내용과 마지막으로 에이전트의 이름을 추가
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(
                    [self.prefix] + self.message_history)),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        name 이 말한 message 를 메시지 내역에 추가
        """
        self.message_history.append(f"{name}: {message}")

<br>

#### `DialogueSimulator`
- `inject` 메서드는 주어진 이름(name)과 메시지(message)로 대화를 시작하고, 모든 에이전트가 해당 메시지를 받도록 함
- `step` 메서드는 다음 발언자를 선택하고, 해당 발언자가 메시지를 보내면 모든 에이전트가 메시지를 받도록 하고, 그리고 현재 단계를 증가

<br>

- **`DialogueAgent`는 개별 에이전트를 나타내며, `DialogueSimulator`는 에이전트들 간의 대화를 조정하고 관리**

In [24]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        # 에이전트 목록을 설정
        self.agents = agents
        # 시뮬레이션 단계를 초기화
        self._step = 0
        # 다음 발언자를 선택하는 함수를 설정
        self.select_next_speaker = selection_function

    def reset(self):
        # 모든 에이전트를 초기화
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        name 의 message 로 대화를 시작
        """
        # 모든 에이전트가 메시지를 받음
        for agent in self.agents:
            agent.receive(name, message)

        # 시뮬레이션 단계를 증가
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. 다음 발언자를 선택
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. 다음 발언자에게 메시지를 전송
        message = speaker.send()

        # 3. 모든 에이전트가 메시지를 받음
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. 시뮬레이션 단계를 증가
        self._step += 1

        # 발언자의 이름과 메시지를 반환
        return speaker.name, message


<br>

#### `DialogueAgentWithTools`
- `DialogueAgent`를 확장하여 도구를 사용할 수 있도록 `DialogueAgentWithTools` 클래스를 정의
- `DialogueAgentWithTools` 클래스는 `DialogueAgent` 클래스를 상속받아 구현
- `send` 메서드는 에이전트가 메시지를 생성하고 반환하는 역할
- `create_openai_tools_agent` 함수를 사용하여 에이전트 체인을 초기화
- 초기화시 에이전트가 사용할 도구(`tools`) 를 정의

In [27]:
from langchain_classic.agents import AgentExecutor, create_openai_tools_agent
from langchain_classic import hub

In [28]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tools,
    ) -> None:
        # 부모 클래스의 생성자를 호출
        super().__init__(name, system_message, model)
        # 주어진 도구 이름과 인자를 사용하여 도구를 로드
        self.tools = tools

    def send(self) -> str:
        """
        메시지 기록에 챗 모델을 적용하고 메시지 문자열을 반환
        """
        prompt = hub.pull("hwchase17/openai-functions-agent")
        agent = create_openai_tools_agent(self.model, self.tools, prompt)
        agent_executor = AgentExecutor(agent=agent, tools=self.tools, verbose=True)
        # AI 메시지를 생성
        message = AIMessage(
            content=agent_executor.invoke(
                {
                    "input": "\n".join(
                        [self.system_message.content]
                        + [self.prefix]
                        + self.message_history
                    )
                }
            )["output"]
        )

        # 생성된 메시지의 내용을 반환
        return message.content

<br>

### 도구 설정

<br>

#### 문서 검색 도구(Retrieval Tool)를 정의



In [29]:
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_classic.document_loaders import TextLoader

In [ ]:
loader1 = TextLoader("data/의대증원반대.txt")
loader2 = TextLoader("data/의대증원찬성.txt")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

docs1 = loader1.load_and_split(text_splitter)
docs2 = loader2.load_and_split(text_splitter)

vector1 = FAISS.from_documents(docs1, OpenAIEmbeddings())
vector2 = FAISS.from_documents(docs2, OpenAIEmbeddings())

doctor_retriever = vector1.as_retriever(search_kwargs={"k": 3})
gov_retriever = vector2.as_retriever(search_kwargs={"k": 3})

<br>

- **langchain 패키지의 tools 모듈에서 retriever 도구를 생성하는 함수**

In [32]:
from langchain_classic.tools.retriever import create_retriever_tool

In [ ]:
doctor_retriever_tool = create_retriever_tool(
    doctor_retriever,
    name="document_search",
    description="This is a document about the Korean Medical Association's opposition to the expansion of university medical schools. "
    "Refer to this document when you want to present a rebuttal to the proponents of medical school expansion.",
)

gov_retriever_tool = create_retriever_tool(
    gov_retriever,
    name="document_search",
    description="This is a document about the Korean government's support for the expansion of university medical schools. "
    "Refer to this document when you want to provide a rebuttal to the opposition to medical school expansion.",
)

<br>

#### 인터넷 검색 도구

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
search = TavilySearchResults(k=3)

<br>

### 각 에이전트가 활용할 수 있는 도구

- `names` 딕셔너리는 토론자의 이름(`prefix name`) 과 각각의 토론 에이전트가 활용할 수 있는 도구를 정의
- `topic` : 토론의 주제를 선정

<br>

#### 1. 문서에 기반한 도구

In [ ]:
names = {
    "Doctor Union(의사협회)": [doctor_retriever_tool],  # 의사협회 에이전트 도구 목록
    "Government(대한민국 정부)": [gov_retriever_tool],  # 정부 에이전트 도구 목록
}

# 토론 주제 선정
topic = "2024 현재, 대한민국 대학교 의대 정원 확대 충원은 필요한가?"

# 토론자를 설명하는 문구의 단어 제한
word_limit = 50

<br>

#### 검색(Search) 기반 도구


In [ ]:
names = {
    "Doctor Union(의사협회)": [search],  # 의사협회 에이전트 도구 목록
    "Government(대한민국 정부)": [search],  # 정부 에이전트 도구 목록
}

# 토론 주제 선정
topic = "2024년 현재, 대한민국 대학교 의대 정원 확대 충원은 필요한가?"
word_limit = 50  # 작업 브레인스토밍을 위한 단어 제한

<br>

#### LLM을 활용하여 주제 설명에 세부 내용 추가하기
- LLM(Large Language Model)을 사용하여 주어진 주제에 대한 설명을 보다 상세하게 만들 수 있음
-  먼저 주제에 대한 간단한 설명이나 개요를 LLM에 입력으로 제공 $\rightarrow$ LLM에게 해당 주제에 대해 더 자세히 설명해줄 것을 요청
-  LLM은 방대한 양의 텍스트 데이터를 학습했기 때문에, 주어진 주제와 관련된 추가적인 정보와 세부 사항을 생성해낼 수 있음
  
  $\rightarrow$ 초기의 간단한 설명을 보다 풍부하고 상세한 내용으로 확장

<br>

1. 주어진 대화 주제(`topic`)와 참가자(`names`)를 기반으로 대화에 대한 설명(`conversation_description`)을 생성

In [ ]:
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""


2. `agent_descriptor_system_message` 는 대화 참가자에 대한 설명을 추가할 수 있다는 내용의 `SystemMessage`

In [ ]:
agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


3. `generate_agent_description` 함수는 각 참가자(`name`)에 대하여 LLM 이 생성한 설명을 생성

In [ ]:
def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a description of {name}, in {word_limit} words or less in expert tone. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else. Answer in KOREAN."""
        ),
    ]
    # ChatOpenAI를 사용하여 에이전트 설명을 생성
    agent_description = ChatOpenAI(temperature=0)(
        agent_specifier_prompt).content
    return agent_description

4. `agent_specifier_prompt` 는 대화 설명과 참가자 이름, 단어 제한(`word_limit`)을 포함하는 `HumanMessage로` 구성
5. ChatOpenAI 모델을 사용하여 `agent_specifier_prompt` 를 기반으로 참가자에 대한 설명(`agent_description`)을 생성

In [ ]:
agent_descriptions = {name: generate_agent_description(name) for name in names}

agent_descriptions

<br>

- 직접 각 토론자의 간략한 입장에 대하여 설명하는 문구를 작성

In [ ]:
agent_descriptions = {
    "Doctor Union(의사협회)": "의사협회는 의료계의 권익을 보호하고 의사들의 이해관계를 대변하는 기관입니다. 의사들의 업무 환경과 안전을 중시하며, 환자 안전과 질 높은 의료 서비스를 제공하기 위해 노력합니다. "
    "지금도 의사의 수는 충분하다는 입장이며, 의대 증원은 필수 의료나 지방 의료 활성화에 대한 실효성이 떨어집니다. 의대 증원을 감행할 경우, 의료 교육 현장의 인프라가 갑작스러운 증원을 감당하지 못할 것이란 우려를 표합니다.",
    "Government(대한민국 정부)": "대한민국 정부는 국가의 행정을 책임지는 주체로서, 국민의 복지와 발전을 책임져야 합니다. "
    "우리나라는 의사수가 절대 부족한 상황이며, 노인인구가 늘어나면서 의료 수요가 급증하고 있습니다. OECD 국가들도 최근 의사수를 늘렸습니다. 또한, 증원된 의사 인력이 필수의료와 지역 의료로 갈 수있도록 튼튼한 의료사고 안정망 구축 및 보상 체계의 공정성을 높이고자 합니다.",
}


<br>

### 전역 System Message 설정
- System message는 대화형 AI 시스템에서 사용자의 입력에 앞서 시스템이 생성하는 메시지
- 이러한 메시지는 대화의 맥락을 설정하고, 사용자에게 각 에이전트의 입장과 목적 을 알려주는 역할
- 효과적인 system message를 작성하면 사용자와의 상호작용을 원활하게 하고, 대화의 질을 높일 수 있음

<br>

#### 프롬프트 설명
- 에이전트의 이름과 설명
- 에이전트는 도구를 사용하여 정보를 찾고 대화 상대방의 주장을 반박
- 에이전트는 출처를 인용해야 하며, 가짜 인용을 하거나 찾아보지 않은 출처를 인용해서는 안 됨
- 에이전트는 자신의 관점에서 말을 마치는 즉시 대화를 중단

<br>


In [1]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}

Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.

Answer in KOREAN.
"""

In [ ]:
agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

In [ ]:
for name, system_message in agent_system_messages.items():
    # 에이전트의 이름을 출력
    print(name)
    # 에이전트의 시스템 메시지를 출력
    print(system_message)

<br>

- `topic_specifier_prompt`를 정의하여 주어진 주제를 더 구체화하는 프롬프트를 생성
  - `temperature` 를 조절하여 더 다양한 주제를 생성

In [ ]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}

        You are the moderator. 
        Please make the topic more specific.
        Please reply with the specified quest in 100 words or less.
        Speak directly to the participants: {*names,}.  
        Do not add anything else.
        Answer in Korean."""
    ),
]

- 구체화된 주제를 생성

In [ ]:
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content
# specified_topic = "정부는 2025년 입시부터 의대 입학정원을 2000명 늘린다고 발표했습니다. 이에 의사단체는 전국에서 규탄집회를 열어 반발하고 있습니다. 의대 정원 확대를 둘러싼 논란 쟁점을 짚어보고, 필수 의료와 지역 의료 해법에 대해서 토론해주세요."

print(f"Original topic:\n{topic}\n")  # 원래 주제를 출력
print(f"Detailed topic:\n{specified_topic}\n")  # 구체화된 주제를 출력

<br>

### 토론 Loop
- **토론 루프는 프로그램의 핵심 실행 부분으로, 주요 작업이 반복적으로 수행되는 곳**
  
  **여기서 주요 작업은 각 에이전트의 메시지 청취 $\rightarrow$ 도구를 활용하여 근거 탐색 $\rightarrow$ 반박 의견 제시 등을 포함**

In [ ]:
agents = [
    DialogueAgentWithTools(
        name=name,
        system_message=SystemMessage(content=system_message),
        model=ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0.2),
        tools=tools,
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

- `select_next_speaker` 함수는 다음 발언자를 선택하는 역할

In [ ]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    # step을 에이전트 수로 나눈 나머지를 인덱스로 사용하여 다음 발언자를 순환적으로 선택.
    idx = (step) % len(agents)
    return idx


<br>

#### 최대 6번의 토론을 실행 (`max_iters=6`)
- `DialogueSimulator` 클래스의 인스턴스인 `simulator`를 생성하며, `agents`와 `select_next_speaker` 함수를 매개변수로 전달
- `simulator.reset()` 메서드를 호출하여 시뮬레이터를 초기화
- `simulator.inject()` 메서드를 사용하여 "Moderator" 에이전트에게 `specified_topic`을 주입
- "Moderator"가 말한 `specified_topic`을 출력
- `n`이 `max_iters`보다 작은 동안 반복:
  - `simulator.step()` 메서드를 호출하여 다음 에이전트의 이름(`name`)과 메시지(`message`)를 가져옴
  - 에이전트의 이름과 메시지를 출력
  - `n`을 1 증가

In [ ]:
max_iters = 6  # 최대 반복 횟수
n = 0

- `DialogueSimulator` 객체를 생성하고, `agents`와 `select_next_speaker` 함수를 전달

In [ ]:
simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)

* 시뮬레이터를 초기 상태로 리셋

In [ ]:
simulator.reset()

- Moderator가 제시한 주제를 출력

In [ ]:
simulator.inject("Moderator", specified_topic)

print(f"(Moderator): {specified_topic}")
print("\n")

In [ ]:
while n < max_iters:
    name, message = (
        simulator.step()
    )  # 시뮬레이터의 다음 단계를 실행하고 발언자와 메시지를 받아옴
    print(f"({name}): {message}")  # 발언자와 메시지를 출력
    print("\n")
    n += 1

<br>

<hr>